In [ ]:
#<img src="Figs/GEOS_Logo.png" width="700"/> 


# Step **X** of **`G2FNL`**: <font color=blue>"centered_moving_avg.ipynb"</font>
#### August 2, 2021  <font color=red>(v. testing)</font> 
##### Jeonghyeop Kim (jeonghyeop.kim@gmail.com)


> quick and dirty \
> (-10 days) the date (+10 days) centered avg. \
> If fewer than 7 days positions for any of side are available, skip the date. \
> Later do the 6-month moving box-car calculation (one - another 6 month before) \


In [2]:
# 1. import python modules
import numpy as np
import pandas as pd
from datetime import datetime
from ismember import ismember



In [3]:
HowManyDays = 10 # before and after the day, how many days would you consider to compute centered-moving avg value?

In [4]:
for i in range(906,907): #906
    
    target_data="no_outliers_"+str(i+1)

    df_GPS=pd.read_csv(target_data, header=None, sep=' ')
    df_GPS.columns=['time','lon','lat','e','n','z','se','sn','sz','corr_en','flag']
    df_GPS['datenum'] = df_GPS.index + 1 #consecutive integer datenum!
    #The algorithm already fills up all the blank time with 0 values
    #Thus one can just define datenum by adding 1 to the index
    #Otherwise, read 'time_vector.dat' to obtain datenum.
    
    df_GPS=df_GPS[['time','datenum','lon','lat','e','n','z','se','sn','sz','corr_en','flag']]
    
    lst = []
    cols = ['time','datenum','lon','lat','e','n','z','se','sn','sz','corr_en','flag']
    

#     for i in range(2):
#     this_column = df.columns[i]
#     df[this_column] = [i, i+1]
    
    for j in range(HowManyDays,len(df_GPS)-HowManyDays): #range(13,14): #
        
        
        df_date=pd.DataFrame(df_GPS.loc[[j]])
        
        df_before=df_GPS.loc[range(j-HowManyDays,j)]
        df_after=df_GPS.loc[range(j+1,j+HowManyDays+1)]
        
        df_nonzero_before = df_before[df_before.flag > 0]
        df_nonzero_after = df_after[df_after.flag > 0]
        
        N_b=len(df_nonzero_before)
        N_a=len(df_nonzero_after)
        
        if N_b >= 8 and N_a >= 8:
            time=int(df_date['time'])
            datenum=int(df_date['datenum'])
            z=float(df_date['z'])
            se=float(df_date['se'])
            sn=float(df_date['sn'])
            sz=float(df_date['sz'])
            corr_en=float(df_date['corr_en'])
            flag=1
            
            if float(df_date['e'])==0: #error here 
                #print(i)
                ebefore=df_nonzero_before['e'].sum()
                eafter=df_nonzero_after['e'].sum()
                
                nbefore=df_nonzero_before['n'].sum()
                nafter=df_nonzero_after['n'].sum()
                
                e_mean = ebefore+eafter
                e_mean = e_mean/(N_b+N_a)
                
                n_mean = nbefore+nafter
                n_mean = n_mean/(N_b+N_a)                
                
                lat=df_nonzero_before['lat'].mean()
                lon=df_nonzero_after['lon'].mean()
                
                #error propagation later.
                
            else:
                
                #print(i)
                ebefore=df_nonzero_before['e'].sum()
                eafter=df_nonzero_after['e'].sum()
                e=df_date['e']
                e_date=float(e)
                
                e_mean = e_date+ebefore+eafter
                e_mean = e_mean/(N_b+N_a+1)
                
                
                nbefore=df_nonzero_before['n'].sum()
                nafter=df_nonzero_after['n'].sum()
                n=df_date['n']
                n_date=float(n)
                
                n_mean = n_date+nbefore+nafter
                n_mean = n_mean/(N_b+N_a+1)
                
                lat=float(df_date['lat'])
                lon=float(df_date['lon'])
            
            #'time','datenum','lon','lat','e','n','z','se','sn','sz','corr_en','flag'
            
            lst.append([time,datenum,lon,lat,e_mean,n_mean,z,se,sn,sz,corr_en,flag])
            
            
        else:
            tooSmall=1 #pass!
            

            
    #SAVE df with centered-avg daily values
    df = pd.DataFrame(lst, columns=cols)
    savefile = "centered_moving_avg_"+str(i+1) #output file 
    df.to_csv(savefile ,header=None, index=None,sep=' ',float_format='%g') #SAVE AS THEY ARE
    

        
        
    
        
        


In [5]:
df_GPS

,time,datenum,lon,lat,e,n,z,se,sn,sz,corr_en,flag
0,20060101,1,0.00,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0
1,20060102,2,0.00,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0
2,20060103,3,0.00,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0
3,20060104,4,0.00,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0
4,20060105,5,0.00,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0
...,...,...,...,...,...,...,...,...,...,...,...,...
5321,20200727,5322,243.92,35.287,0.532473,0.749319,0.031463,0.000697,0.000827,0.002808,0.053765,1
5322,20200728,5323,243.92,35.287,0.533723,0.748869,0.032515,0.000719,0.000843,0.002802,0.088391,1
5323,20200729,5324,243.92,35.287,0.535312,0.749941,0.028686,0.000681,0.000804,0.002667,0.025269,1
5324,20200730,5325,243.92,35.287,0.533898,0.749904,0.023547,0.000685,0.000803,0.002683,0.027569,1
